Lendo o arquivo com os dados

In [1]:
# Imports
import pandas as pd
from sklearn.model_selection import train_test_split
import numpy as np

In [2]:
nome_do_arquivo = 'Market variables/Bitcoin Historical Data (1).csv' 
dados = pd.read_csv(nome_do_arquivo)

nome_do_arquivo_2 = '../dataset_2.csv' 
dados_full = pd.read_csv(nome_do_arquivo_2)

dados_full['Date'] = dados['Date'][::-1].values
dados_full['Change %'] = dados['Change %'][::-1].values

dados = dados_full

dados

,bitcoin_historical_data_Close,bitcoin_historical_data_Open,bitcoin_historical_data_High,bitcoin_historical_data_Low,bitcoin_historical_data_Vol.,btc_issuance_bitcoin_Issuance,btc_miner_rewards_Transaction Fees,btc_miner_rewards_Rewards,btc_issuance_percentage_Issuance,btc_total_addresses_Total,...,btc_transaction_volume_usd_size_adjusted_> $10m,btc_transactions_volume_btc_total_Total Volume,price_trend,SMA_20,STD_20,Upper_Band,Lower_Band,RSI,Date,Change %
0,0.1,0.0,0.1,0.1,80.0,82250.000000,2.831429,82250.000000,1.241429,8.891368e+04,...,0.000000,113100.360204,STABLE,NaN,NaN,NaN,NaN,NaN,07/18/2010,0.00%
1,0.1,0.1,0.1,0.1,570.0,67600.000000,3.000000,67600.000000,1.010000,8.986329e+04,...,0.000000,46194.347143,STABLE,NaN,NaN,NaN,NaN,NaN,07/19/2010,0.00%
2,0.1,0.1,0.1,0.1,260.0,67742.857143,3.288571,67742.857143,1.010000,9.049402e+04,...,0.000000,51427.162041,STABLE,NaN,NaN,NaN,NaN,NaN,07/20/2010,0.00%
3,0.1,0.1,0.1,0.1,580.0,67885.714286,3.577143,67885.714286,1.010000,9.112476e+04,...,0.000000,56659.976939,STABLE,NaN,NaN,NaN,NaN,NaN,07/21/2010,0.00%
4,0.1,0.1,0.1,0.1,2160.0,68028.571429,3.865714,68028.571429,1.010000,9.175549e+04,...,0.000000,61892.791837,STABLE,NaN,NaN,NaN,NaN,NaN,07/22/2010,0.00%
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4849,33901.8,34153.7,34245.8,33423.6,47330.0,3310.714286,89.190000,3310.714286,0.017767,1.228292e+09,...,0.485894,964609.869787,DOWN,29522.550,2782.317334,35087.184667,23957.915333,90.798428,10/27/2023,-0.74%
4850,34082.6,33901.6,34387.8,33867.3,22000.0,2532.142857,65.500000,2532.142857,0.017919,1.228597e+09,...,0.489727,966867.149535,UP,29830.545,2932.771482,35696.087964,23965.002036,91.098024,10/28/2023,0.53%
4851,34526.7,34082.6,34737.9,33937.6,27720.0,1753.571429,41.810000,1753.571429,0.018071,1.228901e+09,...,0.493560,969124.429284,UP,30177.810,3060.832203,36299.474405,24056.145595,91.233275,10/29/2023,1.30%
4852,34477.9,34526.7,34853.3,34089.5,45200.0,975.000000,18.120000,975.000000,0.018223,1.229206e+09,...,0.497394,971381.709033,DOWN,30532.155,3130.650161,36793.455321,24270.854679,89.100867,10/30/2023,-0.14%


Agora renomear as colunas necessárias e mudar e adicionar a variável alvo.

In [3]:
dados = dados.rename(columns={
    'bitcoin_historical_data_Close': 'close',
    'bitcoin_historical_data_Open': 'open',
    'bitcoin_historical_data_High': 'high',
    'bitcoin_historical_data_Low': 'low',
    'bitcoin_historical_data_Vol.': 'vol'
})

Data set:

Os dados do dataset são refetentes aos dados diários do ativo, ou seja, temos o open, high, low, volume e Price_close do ativo depois do fechamento do dia. 
O código a seguir tem a intenção de, com os dados do dia anterior, prever alta ou baixa no dia seguinte. 

Para que nossos dados funcionem no problema em questão, deslocaremos o "Change" para baixo. Inicialmente esse change é uma comparação entre o dia atual e o anterior. Se houve subida, teremos um valor positivo, e caso contrario será negativo. Sendo assim, será necessario deslocar essa coluna pra baixo. Assim será nossa variavel alvo.

Depois de arrumar a variável alvo no código, deslocaremos todas as linhas para baixo com o intúito de fazer a ultima das modificações relevantes nos dados. Como nosso objetivo é prever se o dia de hj, que acabou de começar, vai ser uma alta ou baixa. Para tal, nossos dados atuais serão referentes ao dia de ontem. Por esse motivo que é necessário deslocar a tabela para baixo. Dessa forma os ultimos dados de hj serão referentes ao dia de ontem.

In [4]:
#Isso aqui inverte os dados para que fique igual ao outro dataset. Assim posso reutilizar o
#código a baixo.
dados = dados.iloc[::-1]

# Convertendo a coluna 'Change %' para float e verificando se é positivo ou negativo
dados['Change %'] = dados['Change %'].str.rstrip('%').astype('float')  # Removendo o símbolo de porcentagem e convertendo para float

# Atualizando os valores da coluna 'Change %' com base nas condições
dados['Change %'] = dados['Change %'].apply(lambda x: 1 if x >= 0 else -1)


#Shift na coluna alvo pra que ela fique coerente. O alvo indica oq o algorítimo deve prever.
dados['Change %'] = dados['Change %'].shift(1)

#Aqui damos shift em todo o data set. Isso é feito para que os dados fiquem coerentes. Tipo, no dia 2 da tabela, teremos todas as infos do dia 1.
date_column = dados['Date']
dados = dados.shift(1)
dados['Date'] = date_column
 
#Tirando os valores NaN
dados = dados.dropna()

# Renomear a coluna 'Change %' para 'target'
dados = dados.rename(columns={'Change %': 'predicted'})

# Definir a coluna "Date" como índice
dados.set_index('Date', inplace=True)

colunas_para_ajustar = ['close', 'open', 'high', 'low']  # Substitua pelos nomes reais das colunas no seu DataFrame

# Remover a vírgula e converter para o tipo numérico
# for coluna in colunas_para_ajustar:
#     dados[coluna] = dados[coluna].str.replace(',', '')#.astype(float)


# #Converter valores de volume de str para float
# def convert_to_numeric(value):
#     if 'B' in value:
#         return float(value.replace('B', '')) * 1000000000  # Convertendo para bilhões
#     elif 'M' in value:
#         return float(value.replace('M', '')) * 1000000    # Convertendo para milhões
#     elif 'K' in value:
#         return float(value.replace('K', '')) * 1000       # Convertendo para milhares
#     else:
#         return float(value)

# Aplicar a função para converter os valores na coluna 'vol' para números
# dados['vol'] = dados['vol'].apply(convert_to_numeric)

dados = dados.iloc[::-1]

dados

,close,open,high,low,vol,btc_issuance_bitcoin_Issuance,btc_miner_rewards_Transaction Fees,btc_miner_rewards_Rewards,btc_issuance_percentage_Issuance,btc_total_addresses_Total,...,btc_transaction_volume_usd_size_adjusted_ $1m - $10m,btc_transaction_volume_usd_size_adjusted_> $10m,btc_transactions_volume_btc_total_Total Volume,price_trend,SMA_20,STD_20,Upper_Band,Lower_Band,RSI,predicted
Date,,,,,,,,,,,,,,,,,,,,,
10/25/2010,0.2,0.1,0.2,0.1,19220.0,75728.571429,0.00,75728.571429,0.893064,1.350007e+05,...,0.000000,0.000000,136504.514308,UP,0.105,0.022361,0.149721,0.060279,100.000000,1.0
10/26/2010,0.2,0.2,0.2,0.2,65610.0,74957.142857,0.00,74957.142857,0.881982,1.353939e+05,...,0.000000,0.000000,129181.381473,STABLE,0.110,0.030779,0.171559,0.048441,100.000000,1.0
10/27/2010,0.2,0.2,0.2,0.2,21530.0,74185.714286,0.00,74185.714286,0.870899,1.357872e+05,...,0.000000,0.000000,121858.248638,STABLE,0.115,0.036635,0.188270,0.041730,100.000000,1.0
10/28/2010,0.2,0.2,0.2,0.2,28690.0,73414.285714,0.00,73414.285714,0.859816,1.361804e+05,...,0.000000,0.000000,114535.115803,STABLE,0.120,0.041039,0.202078,0.037922,100.000000,1.0
10/29/2010,0.2,0.2,0.2,0.2,26710.0,72642.857143,0.00,72642.857143,0.848733,1.365737e+05,...,0.000000,0.000000,107211.982968,STABLE,0.125,0.044426,0.213852,0.036148,100.000000,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10/25/2023,34153.7,34497.6,34826.0,33749.9,56620.0,4089.285714,112.88,4089.285714,0.017614,1.227987e+09,...,0.353783,0.482061,962352.590038,DOWN,29225.515,2601.418691,34428.352381,24022.677619,93.607985,-1.0
10/26/2023,33901.8,34153.7,34245.8,33423.6,47330.0,3310.714286,89.19,3310.714286,0.017767,1.228292e+09,...,0.350955,0.485894,964609.869787,DOWN,29522.550,2782.317334,35087.184667,23957.915333,90.798428,1.0
10/27/2023,34082.6,33901.6,34387.8,33867.3,22000.0,2532.142857,65.50,2532.142857,0.017919,1.228597e+09,...,0.348127,0.489727,966867.149535,UP,29830.545,2932.771482,35696.087964,23965.002036,91.098024,1.0


In [5]:
dados.to_csv('dados_w_full_features.csv', index=False)